# AGENTS FILE 🧑‍🏭
FILE FOR TESTING AND DEBUGGING

In [47]:
# Load environment variables FIRST - before any ADK imports!
from dotenv import load_dotenv
import os
import sys
from pathlib import Path

# Add parent directory to path for importing our custom modules
project_root = Path().absolute().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Load .env file from project root IMMEDIATELY
env_path = project_root / ".env"
load_dotenv(dotenv_path=env_path)

# Verify environment variables are loaded
api_key = os.getenv("GOOGLE_API_KEY")
use_vertexai = os.getenv("GOOGLE_GENAI_USE_VERTEXAI", "FALSE")

# CRITICAL: Explicitly set environment variables for ADK
# This ensures ADK can access them when auto-initializing
if api_key:
    os.environ["GOOGLE_API_KEY"] = api_key
if use_vertexai:
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = use_vertexai

# Now import ADK components - AFTER environment is set
from google.adk.agents import Agent
from google.adk.runners import InMemoryRunner
from google.genai import types

# Verify environment configuration
print("=" * 60)
print("🔧 Environment Configuration")
print("=" * 60)

if api_key:
    print(f"✅ GOOGLE_API_KEY: {api_key[:2]}...{api_key[-2:]}")
else:
    print("⚠️  WARNING: GOOGLE_API_KEY not found!")
    print(f"   Looking for .env at: {env_path}")
    print("   Make sure you have created a .env file with GOOGLE_API_KEY=your_key")

if use_vertexai:
    print(f"✅ GOOGLE_GENAI_USE_VERTEXAI: {use_vertexai}")
    if use_vertexai.upper() == "TRUE":
        print("   → Using Google Cloud Vertex AI")
        project_id = os.getenv("GOOGLE_CLOUD_PROJECT")
        location = os.getenv("GOOGLE_CLOUD_LOCATION")
        if project_id:
            print(f"   → Project: {project_id}")
        if location:
            print(f"   → Location: {location}")
    elif use_vertexai.upper() == "FALSE":
        print("   → Using Google AI Studio")
else:
    print("⚠️  WARNING: GOOGLE_GENAI_USE_VERTEXAI not set!")
    print("   → Defaulting to Google AI Studio")

print("=" * 60)

# Import our custom tools
from src.tools import read_cv, list_available_cvs, compare_candidates

print("✅ ADK components and custom tools imported successfully.")
print("✅ Environment variables set for ADK auto-initialization")

🔧 Environment Configuration
✅ GOOGLE_API_KEY: AI...KM
✅ GOOGLE_GENAI_USE_VERTEXAI: FALSE
   → Using Google AI Studio
✅ ADK components and custom tools imported successfully.
✅ Environment variables set for ADK auto-initialization


In [48]:
# Using Auto-initialization - ADK will create client from environment variables
# No need to explicitly create Client - simpler and cleaner!
# Just make sure your .env has:
#   GOOGLE_API_KEY=your_key
#   GOOGLE_GENAI_USE_VERTEXAI=FALSE

print("🌐 ADK will auto-initialize client from environment variables")
print("   → Reading GOOGLE_API_KEY from .env")
print("   → Reading GOOGLE_GENAI_USE_VERTEXAI from .env")

🌐 ADK will auto-initialize client from environment variables
   → Reading GOOGLE_API_KEY from .env
   → Reading GOOGLE_GENAI_USE_VERTEXAI from .env


In [49]:
# Define the CV Analysis Agent with custom tools
# ADK will auto-initialize the client from environment variables

root_agent = Agent(
    name="cv_analysis_agent",
    model="gemini-2.5-flash-lite",
    description="Agent to read and analyze CVs from the dummy_files_for_testing folder.",
    instruction="""
    You are a helpful assistant to the human resources department.
    
    Your capabilities:
    - Read CV files using the read_cv tool
    - List available CVs using the list_available_cvs tool
    - Compare candidates using the compare_candidates tool
    
    When analyzing CVs, provide:
    1. Key technical skills and experience level
    2. Languages spoken and proficiency
    3. Work experience summary
    4. Educational background
    5. Overall assessment and recommendations
    
    Be thorough, professional, and objective in your analysis.""",
    tools=[read_cv, list_available_cvs, compare_candidates]
)

print("✅ CV Analysis Agent defined with custom tools.")

✅ CV Analysis Agent defined with custom tools.


In [50]:
# RUNNER 
runner = InMemoryRunner(agent=root_agent)

print("✅ Runner created.")

✅ Runner created.


In [51]:
## 🧪 Testing Section

**Note:** For detailed tool testing, see `tools/test_debug_tools.ipynb`

Below are examples of using the agent with custom tools.


SyntaxError: invalid syntax (1938605284.py, line 3)

In [52]:
# Example 1: List Available CVs
# Simple query - agent uses list_available_cvs tool

print("🤖 Example 1: Listing available CV files\n")

response = await runner.run_debug(
    "Please list all available CV files and tell me what you found."
)

# Extract the final text response
final_response = None
for event in response:
    if hasattr(event, 'content') and event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, 'text') and part.text:
                final_response = part.text

print("=" * 60)
print("📋 AGENT'S RESPONSE:")
print("=" * 60)
if final_response:
    print(final_response)
else:
    print("⚠️ Agent didn't provide a text response")


🤖 Example 1: Listing available CV files


 ### Created new session: debug_session_id

User > Please list all available CV files and tell me what you found.
cv_analysis_agent > I found the following CV files:

**Text files (.txt):**
* cv_john_doe.txt
* cv_maria_santos.txt

**PDF files (.pdf):**
* cv_john_doe.pdf
* cv_maria_santos.pdf
📋 AGENT'S RESPONSE:
I found the following CV files:

**Text files (.txt):**
* cv_john_doe.txt
* cv_maria_santos.txt

**PDF files (.pdf):**
* cv_john_doe.pdf
* cv_maria_santos.pdf


In [53]:
# Example 2: Analyze Specific CV
# Agent uses read_cv tool and provides analysis

print("🤖 Example 2: Analyzing John Doe's CV\n")

prompt = """
Please read and analyze the CV file 'cv_john_doe.txt' and provide a detailed assessment.

Focus on:
1. Technical skills and experience level
2. Languages spoken
3. Work experience summary
4. Educational background
5. Overall suitability for a Senior Python Developer role requiring:
   - 5+ years Python experience
   - Microservices experience
   - Strong communication skills
"""

response = await runner.run_debug(prompt)

# Extract the final text response
final_response = None
for event in response:
    if hasattr(event, 'content') and event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, 'text') and part.text:
                final_response = part.text

print("=" * 60)
print("📋 AGENT'S ANALYSIS:")
print("=" * 60)
if final_response:
    print(final_response)
else:
    print("⚠️ No text response generated")
    print("\n🔍 Agent executed tools but didn't provide written analysis.")
    print("Try being more explicit: 'Please analyze and WRITE a summary...'")



🤖 Example 2: Analyzing John Doe's CV


 ### Continue session: debug_session_id

User > 
Please read and analyze the CV file 'cv_john_doe.txt' and provide a detailed assessment.

Focus on:
1. Technical skills and experience level
2. Languages spoken
3. Work experience summary
4. Educational background
5. Overall suitability for a Senior Python Developer role requiring:
   - 5+ years Python experience
   - Microservices experience
   - Strong communication skills

cv_analysis_agent > Here's a detailed assessment of John Doe's CV for the Senior Python Developer role:

**1. Technical Skills and Experience Level:**
*   **Python:** Advanced proficiency with 5 years of dedicated experience, aligning perfectly with the requirement.
*   **Microservices:** Explicitly mentioned experience in developing microservices architecture.
*   **Other Relevant Skills:** Experience with Java, Django, Flask, FastAPI, Spring Boot, Git, Docker, Kubernetes, PostgreSQL, and MongoDB. Familiarity with JavaScript, 

In [54]:
# Example 3: Compare Two Candidates
# Agent uses compare_candidates tool and provides comparison

print("🤖 Example 3: Comparing John Doe vs Maria Santos\n")

prompt = """
Please compare John Doe and Maria Santos for a Senior Python Developer position and write a detailed comparison report.

Focus on:
1. Python experience and expertise
2. Language skills
3. Overall suitability

Provide your analysis and recommendation.
"""

response = await runner.run_debug(prompt)

# Extract the final text response
final_response = None
for event in response:
    if hasattr(event, 'content') and event.content and event.content.parts:
        for part in event.content.parts:
            if hasattr(part, 'text') and part.text:
                final_response = part.text

print("=" * 60)
print("📋 COMPARATIVE ANALYSIS:")
print("=" * 60)
if final_response:
    print(final_response)
else:
    print("⚠️ No text response generated")
    print("\n🔍 Agent compared CVs but didn't write analysis.")
    print("Tip: Add 'write a detailed report' to prompt.")


🤖 Example 3: Comparing John Doe vs Maria Santos


 ### Continue session: debug_session_id

User > 
Please compare John Doe and Maria Santos for a Senior Python Developer position and write a detailed comparison report.

Focus on:
1. Python experience and expertise
2. Language skills
3. Overall suitability

Provide your analysis and recommendation.

cv_analysis_agent > ## Candidate Comparison Report: Senior Python Developer Role

This report compares John Doe and Maria Santos based on their CVs, focusing on Python experience, language skills, and overall suitability for a Senior Python Developer position.

**1. Python Experience and Expertise:**

*   **John Doe:**
    *   **Experience:** 5 years of advanced Python experience.
    *   **Expertise:** Proficient with frameworks like Django, Flask, and FastAPI. Demonstrated experience in developing microservices architecture using Python. His current role involves leading development and implementing CI/CD pipelines, indicating a strong gra

In [55]:
# Example 4: Open-ended query - let the agent decide what tools to use

prompt = """
I need to hire a Senior Python Developer. 
Can you help me analyze the available candidates and recommend the best one?

Requirements:
- 5+ years Python experience
- Experience with microservices
- Good communication skills
- Multilingual is a plus
"""

response = await runner.run_debug(prompt)
print(response)



 ### Continue session: debug_session_id

User > 
I need to hire a Senior Python Developer. 
Can you help me analyze the available candidates and recommend the best one?

Requirements:
- 5+ years Python experience
- Experience with microservices
- Good communication skills
- Multilingual is a plus

cv_analysis_agent > Now I will analyze the CVs of John Doe and Maria Santos based on your requirements.

**John Doe Analysis:**

*   **Python Experience:** Advanced (5 years experience). Meets the requirement.
*   **Microservices Experience:** Explicitly mentioned experience in developing microservices architecture. Meets the requirement.
*   **Communication Skills:** Mentioned as "strong communication skills across international teams" in his summary. His role involved collaborating with Spanish-speaking clients. Meets the requirement.
*   **Multilingual:** English (Native), Spanish (Fluent), French (Intermediate). This is a plus.

**Maria Santos Analysis:**

*   **Python Experience:** Inte

In [ ]:
# TODO: AGENT TO MATCH THE CANDIDATE TO THE JOB DESCRIPTION
# TODO: AGENT TO CREATE AN ASSESSMENT FOR THE CANDIDATE (CODE INTERVIEW)
# TODO: AGENT TO CREATE AN ASSESSMENT FOR THE CANDIDATE (LANGUAGE TEST)
# TODO: AGENT TO SCHEDULE THE CANDIDATE FOR THE LIVE INTERVIEW